In [ ]:
%run ../utils/access_store_ui.ipynb
display(get_access_store_ui('../'))

In [ ]:
%run ./s3_vs_setup.ipynb

# Create S3 connection

In [ ]:
import pyexasol
import pathlib
from exasol.nb_connector.connections import open_pyexasol_connection
from exasol.nb_connector import bfs_utils

In [ ]:
sql = """
CREATE CONNECTION S3_CONNECTION
  TO ''
  USER ''
  IDENTIFIED BY '{
      "awsRegion": "eu-central-1", 
      "s3Bucket": "ai-lab-example-data-s3" 
  }';
"""

with open_pyexasol_connection(ai_lab_config) as conn:
    conn.execute(sql)

In [ ]:
# put edml to bucket fs
bfs_bucket = open_bucketfs_connection(ai_lab_config)
bfs_path = bfs_utils.put_file(bfs_bucket, pathlib.Path("reuters-edml.json"))

In [ ]:
bfs_path.as_udf_path()

In [ ]:
sql = """
CREATE VIRTUAL SCHEMA {schema_name!i}_VS USING {schema_name!i}.S3_FILES_ADAPTER WITH
    CONNECTION_NAME = 'S3_CONNECTION'
    MAPPING         = {map_file!s};
"""

with open_pyexasol_connection(ai_lab_config) as conn:
    conn.execute(sql, query_params={
        "schema_name": ai_lab_config.db_schema,
        "map_file": bfs_path.as_udf_path()
    })

In [ ]:
sql = """
select * from AI_LAB_VS.NEWS
limit 10
"""

with open_pyexasol_connection(ai_lab_config) as conn:
    res = conn.execute(sql)
    for r in res:
        print(r)